# Test notebook

In [1]:
library(ArchR)
library(org.Ss.eg.db)
library(SuscrofaTxdb.11.108.july)
library(Repitools)
library(openxlsx)
library(topGO)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [3]:
addArchRThreads(2)

Setting default number of Parallel threads to 2.



In [4]:
load(file = "/home/adufour/work/rds_storage/omics/archr_all_v7_stemcells.RData")

In [5]:
metadata <- archrproj_sub@cellColData

In [6]:
metadata$cell <- rownames(metadata)

In [7]:
write.xlsx(metadata, file="/home/adufour/work/table/stemcells_metadata.xlsx", rowNames=TRUE)

# peakDF table

In [16]:
peak_grange <- archrproj_sub@peakSet

In [17]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), "_", start(.), "_", end(.))}

In [18]:
names(peak_grange) <- NULL

In [19]:
peak_DF <- annoGR2DF(peak_grange)

In [20]:
peak_DF <- peak_DF[,c('peakName', 'chr', 'start', 'end', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 
          'distToGeneStart', 'nearestGene', 'peakType', 'distToTSS', 'nearestTSS', 'GC')]

In [21]:
openxlsx::write.xlsx(peak_DF, file="/home/adufour/work/table/peakDF_stemcells.xlsx", rowNames=FALSE)

# P2G

In [22]:
scriptPath <- "/home/adufour/work/scScalpChromatin"
source(paste0(scriptPath, "/misc_helpers.R"))
source(paste0(scriptPath, "/matrix_helpers.R"))
source(paste0(scriptPath, "/plotting_config.R"))
source(paste0(scriptPath, "/archr_helpers.R"))

Warning message:
“le package ‘dplyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘tidyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘RcppAlgos’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘ggrastr’ a été compilé avec la version R 4.2.3”


In [23]:
corrCutoff <- 0.5
p2gGR <- getP2G_GR(archrproj_sub, corrCutoff=corrCutoff)

In [24]:
p2geneDF <- Repitools::annoGR2DF(p2gGR)

In [26]:
p2geneDF <- p2geneDF[,c('peakName', 'chr', 'start', 'end', 'Correlation', 'FDR', 'VarQATAC', 'VarQRNA', 'symbol')]

In [27]:
openxlsx::write.xlsx(p2geneDF, file="/home/adufour/work/table/P2G_stemcells.xlsx", rowNames=FALSE)

# export DAG

In [28]:
markersPeaks <- readRDS(file = "/home/adufour/work/rds_storage/omics/markerPeaks_stemcells.rds")

In [29]:
markerPeaksList <- getMarkers(markersPeaks, cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1")

In [30]:
markerPeaksList$C6$peakName <- paste0(markerPeaksList$C6$seqnames, "_", markerPeaksList$C6$start, "_", markerPeaksList$C6$end)
markerPeaksList$C2$peakName <- paste0(markerPeaksList$C2$seqnames, "_", markerPeaksList$C2$start, "_", markerPeaksList$C2$end)
markerPeaksList$C4$peakName <- paste0(markerPeaksList$C4$seqnames, "_", markerPeaksList$C4$start, "_", markerPeaksList$C4$end)

In [31]:
merge_C6 <- merge(markerPeaksList$C6, p2geneDF, by="peakName", all.x=TRUE)
merge_C2 <- merge(markerPeaksList$C2, p2geneDF, by="peakName", all.x=TRUE)
merge_C4 <- merge(markerPeaksList$C4, p2geneDF, by="peakName", all.x=TRUE)

In [32]:
merge_C6 <- merge(merge_C6, peak_DF, by="peakName")
merge_C2 <- merge(merge_C2, peak_DF, by="peakName")
merge_C4 <- merge(merge_C4, peak_DF, by="peakName")

In [33]:
merge_C6$P2G <- ifelse(is.na(merge_C6$symbol), "FALSE", "TRUE")
merge_C2$P2G <- ifelse(is.na(merge_C2$symbol), "FALSE", "TRUE")
merge_C4$P2G <- ifelse(is.na(merge_C4$symbol), "FALSE", "TRUE")

In [34]:
merge_C6 <- merge_C6[,c('peakName', 'seqnames', 'start', 'end', 'Log2FC', 'FDR.x', 'MeanDiff', 'P2G', 'Correlation', 
                        'symbol', 'score', 'nearestGene', 'peakType')]
merge_C2 <- merge_C2[,c('peakName', 'seqnames', 'start', 'end', 'Log2FC', 'FDR.x', 'MeanDiff', 'P2G', 'Correlation', 
                        'symbol', 'score', 'nearestGene', 'peakType')]
merge_C4 <- merge_C4[,c('peakName', 'seqnames', 'start', 'end', 'Log2FC', 'FDR.x', 'MeanDiff', 'P2G', 'Correlation', 
                        'symbol', 'score', 'nearestGene', 'peakType')]

In [35]:
wb <- createWorkbook("DAG")

In [36]:
addWorksheet(wb, "DAG_epi")
addWorksheet(wb, "DAG_pESC-C1")
addWorksheet(wb, "DAG_pESC-C2")

In [37]:
writeData(wb, "DAG_epi", merge_C6)
writeData(wb, "DAG_pESC-C1", merge_C2)
writeData(wb, "DAG_pESC-C2", merge_C4)

In [38]:
saveWorkbook(wb,"/home/adufour/work/table/DAG_stemcells.xlsx",overwrite = TRUE)

# export DEG

In [39]:
markersGS <- readRDS(file = "/home/adufour/work/rds_storage/omics/markerExpression_stemcells.rds")

In [40]:
markerList <- getMarkers(markersGS, cutOff = "FDR <= 0.01 & Log2FC >= 1.25")

In [41]:
RNA_epi <- markerList$epi[,c('name', 'Log2FC', 'FDR', 'MeanDiff')]
RNA_pESC_C1 <- markerList$pESC_C1[,c('name', 'Log2FC', 'FDR', 'MeanDiff')]
RNA_pESC_C2 <- markerList$pESC_C2[,c('name', 'Log2FC', 'FDR', 'MeanDiff')]

In [42]:
wb <- createWorkbook("DEG")

In [43]:
addWorksheet(wb, "DEG_epi")
addWorksheet(wb, "DEG_pESC-C1")
addWorksheet(wb, "DEG_pESC-C2")

In [44]:
writeData(wb, "DEG_epi", RNA_epi)
writeData(wb, "DEG_pESC-C1", RNA_pESC_C1)
writeData(wb, "DEG_pESC-C2", RNA_pESC_C2)

In [45]:
saveWorkbook(wb,"/home/adufour/work/table/DEG_stemcells.xlsx",overwrite = TRUE)

# topGo enrich

In [46]:
calcTopGo <- function(
    allGenes, interestingGenes=NULL, pvals=NULL, geneSel=NULL,
    nodeSize=5, ontology="BP",
    alg="weight01", stat="fisher", topNodes=1000
    ){
    # Calculate GO term enrichments using topGO on provided data
    # https://bioconductor.org/packages/release/bioc/vignettes/topGO/inst/doc/topGO.pdf
    ############################################################
    # allGenes: vector of genenames to be used in GO term search. Expects gene 'symbol' 
    # interestingGenes: predefined list of 'instersting' genes. Incompatible with supplying pvalues.
    # geneSel: function for selecting 'interesting' genes. Can only really be a p-value cutoff...
    # pvals: vector of pvalues corresponding to geneList. If not provided, will assign everything to 1
    # nodeSize: will prune terms that have less than nodeSize number of genes
    # ontology: which GO ontology to use (MF, BP, CC)
    # alg: algorithm to be used for testing GO terms (topGO default is 'weight01')
    # stat: test statistic to use for significant GO terms
    # topNodes: how many GO terms to return in result table

    # Prepare geneList as expected for topGO (i.e. value vector with names of genes)
    if(!is.null(interestingGenes)){
        message(sprintf("Running GO enrichments with %s genes in universe of %s...", 
            length(interestingGenes), length(allGenes)))
        geneList <- factor(as.integer(allGenes %in% interestingGenes))
        names(geneList) <- allGenes
        # Create topGOdata object
        GOdata <- suppressMessages(new(
            "topGOdata",
            ontology = ontology,
            allGenes = geneList,
            annot = annFUN.org, mapping = "org.Ss.eg.db", ID = "symbol",
            nodeSize = nodeSize
            ))
    }else{
        geneList <- pvals
        names(geneList) <- allGenes
        message(sprintf("Running GO enrichments with %s genes in universe of %s...", 
            sum(geneSel(geneList)), length(allGenes)))
        GOdata <- suppressMessages(new(
            "topGOdata",
            ontology = ontology,
            allGenes = geneList,
            geneSel = geneSel,
            annot = annFUN.org, mapping = "org.Ss.eg.db", ID = "symbol",
            nodeSize = nodeSize
            ))
    }

    # Test for enrichment using Fisher's Exact Test
    GOresult <- suppressMessages(runTest(GOdata, algorithm=alg, statistic=stat))
    GenTable(GOdata, pvalue=GOresult, topNodes=topNodes, numChar=1000)
}

In [47]:
embryo <- readRDS("/home/adufour/work/rds_storage/omics/allcell_seurat_obj.rds")

In [48]:
go_cluster_epi <- calcTopGo(rownames(embryo@assays$RNA@counts), interestingGenes=RNA_epi$name, nodeSize=5, ontology="BP")
go_cluster_pESC_C1 <- calcTopGo(rownames(embryo@assays$RNA@counts), interestingGenes=RNA_pESC_C1$name, nodeSize=5, ontology="BP")
go_cluster_pESC_C2 <- calcTopGo(rownames(embryo@assays$RNA@counts), interestingGenes=RNA_pESC_C2$name, nodeSize=5, ontology="BP")

Running GO enrichments with 665 genes in universe of 35670...

Running GO enrichments with 316 genes in universe of 35670...

Running GO enrichments with 1547 genes in universe of 35670...



In [49]:
write.xlsx(go_cluster_pESC_C1, file="/home/adufour/work/table/deg_topgo_stemcells.xlsx", sheetName="deg_pESC-C1", append=TRUE, row.names=FALSE)
write.xlsx(go_cluster_epi, file="/home/adufour/work/table/deg_topgo_stemcells.xlsx", sheetName="deg_epi", append=TRUE, row.names=FALSE)
write.xlsx(go_cluster_pESC_C2, file="/home/adufour/work/table/deg_topgo_stemcells.xlsx", sheetName="deg_pESC-C2", append=TRUE, row.names=FALSE)

Warning message:
“Please use 'rowNames' instead of 'row.names'”
Warning message:
“Please use 'rowNames' instead of 'row.names'”
Warning message:
“Please use 'rowNames' instead of 'row.names'”


In [50]:
wb <- createWorkbook("DEG_GO")

In [51]:
addWorksheet(wb, "DEG_epi")
addWorksheet(wb, "DEG_pESC-C1")
addWorksheet(wb, "DEG_pESC-C2")

In [52]:
writeData(wb, "DEG_epi", go_cluster_epi)
writeData(wb, "DEG_pESC-C1", go_cluster_pESC_C1)
writeData(wb, "DEG_pESC-C2", go_cluster_pESC_C2)

In [53]:
saveWorkbook(wb,"/home/adufour/work/table/deg_topgo_embryo.xlsx",overwrite = TRUE)

In [54]:
merge_C2 <- merge_C2[merge_C2$peakType == "Promoter",]
merge_C2 <- merge_C2[merge_C2$P2G == "FALSE",]

merge_C6 <- merge_C6[merge_C6$peakType == "Promoter",]
merge_C6 <- merge_C6[merge_C6$P2G == "FALSE",]

merge_C4 <- merge_C4[merge_C4$peakType == "Promoter",]
merge_C4 <- merge_C4[merge_C4$P2G == "FALSE",]

In [55]:
peak_reference <- peak_DF[peak_DF$peakType == "Promoter",]

In [56]:
merge_C2 <- calcTopGo(unique(peak_reference$nearestGene), interestingGenes=merge_C2$nearestGene, nodeSize=5, ontology="BP")
merge_C6 <- calcTopGo(unique(peak_reference$nearestGene), interestingGenes=merge_C6$nearestGene, nodeSize=5, ontology="BP")
merge_C4 <- calcTopGo(unique(peak_reference$nearestGene), interestingGenes=merge_C4$nearestGene, nodeSize=5, ontology="BP")

Running GO enrichments with 823 genes in universe of 14787...

Running GO enrichments with 2010 genes in universe of 14787...

Running GO enrichments with 708 genes in universe of 14787...



In [57]:
wb <- createWorkbook("DAG")

In [58]:
addWorksheet(wb, "DAG_epi")
addWorksheet(wb, "DAG_pESC-C1")
addWorksheet(wb, "DAG_pESC-C2")

In [59]:
writeData(wb, "DAG_epi", merge_C6)
writeData(wb, "DAG_pESC-C1", merge_C2)
writeData(wb, "DAG_pESC-C2", merge_C4)

In [60]:
saveWorkbook(wb,"/home/adufour/work/table/dag_topgo_promoter_stemcells.xlsx",overwrite = TRUE)

# motif

In [61]:
motifsUp <- peakAnnoEnrichment(
    seMarker = markersPeaks,
    ArchRProj = archrproj_sub,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
)

ArchR logging to : ArchRLogs/ArchR-peakAnnoEnrichment-174eb5741c5329-Date-2024-06-10_Time-13-27-43.log
If there is an issue, please report to github with logFile!

2024-06-10 13:27:58 : Computing Enrichments 1 of 3, 0.24 mins elapsed.

2024-06-10 13:28:01 : Computing Enrichments 2 of 3, 0.292 mins elapsed.

2024-06-10 13:28:04 : Computing Enrichments 3 of 3, 0.347 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-peakAnnoEnrichment-174eb5741c5329-Date-2024-06-10_Time-13-27-43.log



In [62]:
motif_df <- assays(motifsUp)[["mlog10Padj"]]

In [63]:
motif_df$motif_name <- rownames(motif_df)

In [64]:
motif_df$motif_name <- gsub("None ", "", motif_df$motif_name)

In [65]:
motif_correspondance = read.csv('/home/adufour/work/table/motif_correspondance.csv', header = FALSE)
colnames(motif_correspondance) <- c("motif_name", "correspondance")
head(motif_correspondance)

,motif_name,correspondance
,<chr>,<chr>
1,jaspar__MA0659.3,metacluster_22.4
2,kznf__ZNF264_Imbeault2017_OM_RCADE,metacluster_0.1
3,kznf__ZNF264_Imbeault2017_RP_RCADE,metacluster_0.1
4,jaspar__MA0467.2,metacluster_0.2
5,jaspar__MA0648.1,metacluster_0.2
6,jaspar__MA0682.2,metacluster_0.2


In [66]:
merged_motif <- merge(motif_df, motif_correspondance, by=c("motif_name"), all.x= TRUE)
merged_motif <- merged_motif %>% 
    mutate(correspondance = coalesce(correspondance,motif_name))
head(merged_motif)

,motif_name,C2,C4,C6,correspondance
,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,bergman__Adf1,0.000000,0.000000,2.179057,bergman__Adf1
2,bergman__Aef1,15.926257,4.255657,0.000000,bergman__Aef1
3,bergman__ap,0.000000,88.560157,0.000000,bergman__ap
4,bergman__bab1,2.046457,0.000000,0.000000,metacluster_145.8
5,bergman__bcd,0.000000,41.663457,0.000000,bergman__bcd
6,bergman__bin,1.036257,0.000000,0.000000,bergman__bin


In [67]:
merged_motif$max <- apply(merged_motif[,4:ncol(merged_motif)-2], 1, max)

In [68]:
merged_motif <- merged_motif[order(merged_motif$correspondance, merged_motif$max, decreasing = TRUE),]

In [69]:
merged_motif <- merged_motif[!duplicated(merged_motif$correspondance),]

In [70]:
motif2tf <- readRDS("/home/adufour/work/rds_storage/motif2tf.rds")

In [71]:
motif2tf <- as.data.frame(motif2tf[,c("#motif_id", "gene_name")])

In [72]:
colnames(motif2tf) <- c("motif_id", "gene_name")

In [73]:
merged_df <- merge(merged_motif, motif2tf, by.x = "correspondance", by.y = "motif_id", all.x = TRUE)

In [74]:
result <- merged_df %>%
  group_by(correspondance) %>%
  summarise(gene_list = paste(unique(gene_name), collapse = ", "))

In [75]:
result <- result[order(merged_motif$correspondance),]

In [76]:
merged_motif$TF <- result$gene_list

In [77]:
merged_motif <- merged_motif[,c("correspondance", "motif_name", "C2", "C4", "C6", "max", "TF")]

In [78]:
colnames(merged_motif) <- c("motif_group", "motif_name", "pESC-C1", "pESC-C2", "EPI", "max", "TF")

In [79]:
head(merged_motif)

,motif_group,motif_name,pESC-C1,pESC-C2,EPI,max,TF
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
18239,yetfasco__YPR086W_1327,yetfasco__YPR086W_1327,0.0000000,0.000000,0.2827575,0.000000,GTF2B
18238,yetfasco__YPR065W_1396,yetfasco__YPR065W_1396,0.0000000,19.542857,0.0000000,19.542857,NA
18237,yetfasco__YPR054W_1875,yetfasco__YPR054W_1875,0.6731575,1.332257,0.0000000,1.332257,NA
18234,yetfasco__YPR009W_2236,yetfasco__YPR009W_2236,0.0000000,0.000000,0.6237575,0.000000,NA
18233,yetfasco__YPR008W_1425,yetfasco__YPR008W_1425,0.0000000,0.000000,12.8882575,0.000000,NA
18231,yetfasco__YPL230W_509,yetfasco__YPL230W_509,0.0000000,24.514157,0.0000000,24.514157,NA


In [80]:
write.xlsx(merged_motif, file="/home/adufour/work/table/motif_stemcells2.xlsx", rowNames=FALSE)